In [1]:
from IPython.display import display
import math
import time
from paradim_parameters import PARADIM
from paradim_parameters import PARADIM_UI
from paradim_utils import GetPbs, getStatus, isfloat
from paradim_connect import ShellHandler
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

<IPython.core.display.Javascript object>

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><img src='HubzeroLogo.png' style='position:absolute;right:20px;width:110px;height:32px'><h1 style="text-align:center;padding-top:3px">Calculating UV/Vis spectra on GaAs</h1></div>    

In this tutorial we will see how to calculate the band structures and the optical absorption spectra of
semiconductors. This tutorial we will perform calculations at the Maryland Advanced Research Computing Center
(MARCC), In order to connect to MARCC we need to use the `two-factor authentication' protocol.

For this you will need:
- The username sent to you by MARCC via email
- The password sent to you by MARCC via email
- The Google Authenticator Token

Please fill out all the fields below and press the 'Connect' Button, if a connection is succesfull stablished the message "Connection created successfully" would be displayed.

If you get the message "ERROR: A new connection is required" on any of the steps below, you must update the information with a new google authenticator token, and press the button connect again.

Each time this notebook is loaded, this would assign a new working folder to store results. If you know the folder name containing previous results and skip some of the steps, please change it.

In [2]:
def CreateConnection( event ):    
    global PARADIM_UI, PARADIM
    try :
        PARADIM_UI['s0']['status'].value = "Connecting..."            
        PARADIM['SESSION'] = ShellHandler( PARADIM_UI['s0']['code'].value, 
                                           PARADIM_UI['s0']['user'].value, 
                                           PARADIM_UI['s0']['pwd'].value, 
                                           PARADIM['SERVER'], 
                                           PARADIM['PORT'])
        folder_name = PARADIM_UI['s0']['folder'].value
        if PARADIM['SESSION'] is not None and PARADIM['SESSION'].is_active():                
            stdin, stdout, stderr, command = PARADIM['SESSION'].execute("mkdir $HOME/" + folder_name);
            stdin, stdout, stderr, command = PARADIM['SESSION'].execute("cd $HOME/" + folder_name);            
            PARADIM_UI['s0']['status'].value = "Connection created successfully"
        else:
            PARADIM_UI['s0']['status'].value = "ERROR: There has been an issue with the connection"            
    except:
        PARADIM_UI['s0']['status'].value = "ERROR: There has been an issue with the authentication"
        
PARADIM_UI['s0']['button'].on_click(CreateConnection)
display(PARADIM_UI['s0']['display'])

Group(children=(HTML(value="<p   style='background-color: #DCDCDC; font-size: 150%; padding: 5px'>MARCC Creden…

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><img src='HubzeroLogo.png' style='position:absolute;right:20px;width:110px;height:32px'><h1 style="text-align:center;padding-top:3px">Step 1: Crystal information</h1></div>

We consider GaAs this tutorial as a common semiconductor. We will start with a simple total energy calculation for GaAs in the diamond structure. In order to proceed we first need pseudopotentials, we need one pseudopotential for each atomic species (the pseudopotential describes the atomic nucleus and the electrons except the outermost (valence) shell). The QE pseudopotential libraries can be found at http://www.quantum-espresso.org/pseudopotentials , and will be downloaded in the working folder.

All the parameters in this input file have been optimized separately. the commands executed in the server are shown in the stdin tab, simulation output in the stdout and any error in the stderr tab. Press the "Calculate Self Consistency" button to start the simulation, the Job will be queued, run and then marked as 'C' when the job is complete

In [3]:
def CalculateSelfConsistency ( event ):
    global PARADIM_UI, PARADIM
    if PARADIM['SESSION'] is not None and PARADIM['SESSION'].is_active():        
        id_name = PARADIM['TUTORIAL_NAME'] + '_scf'
        pbs_file = id_name + ".pbs"
        PARADIM['SESSION'].execute("wget http://www.quantum-espresso.org/upf_files/Ga.pz-bhs.UPF", PARADIM_UI['s1']);
        PARADIM['SESSION'].execute("wget http://www.quantum-espresso.org/upf_files/As.pz-bhs.UPF", PARADIM_UI['s1']);
        inputdeck = PARADIM_UI['s1']['input'].value.replace('\n', '\\n')
        PARADIM['SESSION'].execute("printf \"" + inputdeck + "\" > " + id_name + ".in", PARADIM_UI['s1']);
        PARADIM['SESSION'].execute("printf \"" + GetPbs(PARADIM['PBS_TEMPLATE'], id_name).replace('\n', '\\n') + "\" > " + pbs_file, PARADIM_UI['s1']);
        stdin, stdout, stderr, command = PARADIM['SESSION'].execute("qsub " + pbs_file, PARADIM_UI['s1']);
        try:
            code = stdout[len(stdout)-1].strip('\n')
            if (code.isdigit()):
                PARADIM_UI['s1']['job_id'].value = code
                status = 'Q'
                while status != 'E' and status != 'X' and status != 'C' and status != 'O' :
                    status, response, command = getStatus( PARADIM['SESSION'], PARADIM_UI['s1']['job_id'].value, PARADIM_UI['s1'] )
                    PARADIM_UI['s1']['status'].value = status
                    time.sleep(2)                    
                PARADIM['SESSION'].execute("cat " + id_name + ".out", PARADIM_UI['s1']);
            else : 
                PARADIM_UI['s1']['job_id'].value = "Error with the PBS submittion"
        except:
            PARADIM_UI['s1']['job_id'].value = "Error with the PBS submittion"
    else :
        print ("ERROR: A new connection is required")


PARADIM_UI['s1']['button'].on_click(CalculateSelfConsistency)
display (PARADIM_UI['s1']['display'])



<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><img src='HubzeroLogo.png' style='position:absolute;right:20px;width:110px;height:32px'><h1 style="text-align:center;padding-top:3px">Step 2: Calculate Band Structures</h1></div>

Then, we want to calculate the band structure. This calculation is non self-consistent, in the sense that we use values for the ground-state electron density, Hartree, and exchange and correlation potentials. In a non self-consistent calculation the code pw.x determines the Kohn-Sham eigenfunctions and eigenvalues without upgrading the Kohn-Sham Hamiltonian at every step. This is achieved by using the keyword calculation = 'bands' and by specifying the k-points for which we want the eigenvalues:

In this input file the keyword tpiba_b after K_POINTS specifies that we want pw.x to generate a path going through the points specified in the list. The following number (3) is the number of vertices, and the integer following the coordinates (20) is the number of points in each segment. So in this case we will have 20 points from $L = (1/2,1/2,1/2)2\pi/a$ to $\Gamma$ = (0,0,0) and 20 points from $\Gamma=(0,0,0)$ to $X=(1,0,0)2\pi/a$. The points are given in Cartesian coordinates and in units of $2\pi/a$. In this input file we also specify the number of bands that we want to calculate, we are setting nbnd = 8.

Press the "Calculate Bandstructure" button to start the simulation, the job will be queued, run and then marked as 'C' when is complete

In [4]:
def CalculateBandStructure ( event ):
    global PARADIM_UI, PARADIM
    if PARADIM['SESSION'] is not None and PARADIM['SESSION'].is_active():        
        id_name = PARADIM['TUTORIAL_NAME'] + '_bands'
        pbs_file = id_name + ".pbs"
        inputdeck = PARADIM_UI['s2']['input'].value.replace('\n', '\\n')
        PARADIM['SESSION'].execute("printf \"" + inputdeck + "\" > " + id_name + ".in", PARADIM_UI['s2']);
        PARADIM['SESSION'].execute("printf \"" + GetPbs(PARADIM['PBS_TEMPLATE'], id_name).replace('\n', '\\n') + "\" > " + pbs_file, PARADIM_UI['s2']);
        stdin, stdout, stderr, command = PARADIM['SESSION'].execute("qsub " + pbs_file, PARADIM_UI['s2']);
        try:
            code = stdout[len(stdout)-1].strip('\n')
            if (code.isdigit()):
                PARADIM_UI['s2']['job_id'].value = code
                status = 'Q'
                while status != 'E' and status != 'X' and status != 'C' and status != 'O' :
                    status, response, command = getStatus( PARADIM['SESSION'], PARADIM_UI['s2']['job_id'].value, PARADIM_UI['s2'] )
                    PARADIM_UI['s2']['status'].value = status
                    time.sleep(2)
                stdin, stdout, stderr, command = PARADIM['SESSION'].execute("cat " + id_name + ".out", PARADIM_UI['s2']);
            else : 
                PARADIM_UI['s2']['job_id'].value = "Error with the PBS submittion"
        except:
            PARADIM_UI['s2']['job_id'].value = "Error with the PBS submittion"
    else :
        print ("ERROR: A new connection is required")

    
PARADIM_UI['s2']['button'].on_click(CalculateBandStructure)
display (PARADIM_UI['s2']['display'])


<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><img src='HubzeroLogo.png' style='position:absolute;right:20px;width:110px;height:32px'><h1 style="text-align:center;padding-top:3px">Step 4: Extract Bandstructures</h1></div>

For each k-point in the input file, we have the coordinates of the point and the calculated eigenvalues in eV, in this case we requested 8 bands. In order to plot the bands along the chosen path, we must extract these eigenvalues, and calculate the distance covered as we move along the path $L \longrightarrow \Gamma \longrightarrow X$.

* ExtractKPoints: Extract all k-points and their respective eigenvalues
* CalculatePath: Calculates the distance of each k_point in the path
* CreateDataPlot: Creates scattered lines along each band
* CreateAdditionalTics: Includes labels for known symmetry points.

Press the "Extract Bandstructure" button to start the processing, the bandstructures will be visualized using the plotly library

In [5]:
def ExtractKPoints ( stdout ):        
    l = len(stdout)
    i=0
    k_points = []
    while i<l:
        if ' k = ' in stdout[i]: #new k point found
            point = {}
            point['k'] = [float(s) for s in stdout[i].split() if isfloat(s)]
            i = i+2
            point['v'] = [float(s) for s in stdout[i].split() if isfloat(s)]
            point['total'] = len(point['v'])
            k_points.append(point)
        i = i+1
    return k_points

def CalculatePath ( k_points ):        
    path_len = 0
    for i in range(len(k_points)):
        if i > 0:
            p1= k_points[i-1]['k']
            p2= k_points[i]['k']
            path_len = path_len + math.sqrt(math.pow(p2[0]-p1[0],2)+math.pow(p2[1]-p1[1],2)+math.pow(p2[2]-p1[2],2))                                                
        k_points[i]['p'] = path_len
    x_points = [k['p'] for k in k_points]
    return k_points, x_points

def CreateDataPlot( k_points, x_points, ezero, total_bands ):
    data = []
    emin = 0
    emax = 0
    for k in range(total_bands):
        y_points = [p['v'][k]-ezero for p in k_points]
        emax = max(emax, max(y_points))
        emin = min(emin, min(y_points))
        if all(v > 0 for v in y_points):
            band_color = 'rgb(205, 12, 24)'
        else:
            band_color = 'rgb(22, 96, 167)'
        trace = go.Scatter(
            x = x_points,
            y = y_points,
            mode = 'lines',
            line = dict(color = (band_color)))
        data.append(trace)
    return data, emax, emin

def CreateAdditionalTics( data, k_points, labels, emax, emin ):
    ticktext=[]
    tickvals=[]
    for key, value in labels.items():                    
        step = 0
        for k in k_points:
            if (k['k'][0]==value[0] and k['k'][1]==value[1] and k['k'][2]==value[2]):
                step = k['p']
        trace = go.Scatter(
            x=[step, step],
            y=[emin, emax],
            mode="lines",
            line=dict(color="#111111", width=1),
            showlegend=False)
        data.append(trace)
        ticktext.append(key)
        tickvals.append(step)
    return data, ticktext, tickvals



def VisualizeBandStructure ( event ):        
    global PARADIM_UI, PARADIM
    if PARADIM['SESSION'] is not None and PARADIM['SESSION'].is_active():
        labels = {"L":[0.5,0.5,0.5], "G":[0.0, 0.0, 0.0], "X":[1.0, 0.0, 0.0]}
        ezero = 6.2057  
        PARADIM_UI['s3']['output'].clear_output()
        with PARADIM_UI['s3']['output']:            
            id_name = PARADIM['TUTORIAL_NAME'] + '_bands'
            stdin, stdout, stderr, command = PARADIM['SESSION'].execute("cat " + id_name + ".out");
            k_points = ExtractKPoints(stdout)
            PARADIM_UI['s3']['input'].value = '\n'.join([str(c) + ' ' + str(k['v']) for c,k in enumerate(k_points)])
            k_points, x_points = CalculatePath(k_points)
            data, emax, emin = CreateDataPlot(k_points, x_points, ezero, k_points[0]['total'])
            data, ttext, tvals = CreateAdditionalTics(data, k_points, labels, emax, emin)
            layout = go.Layout(title='GaAs Bandstructure',
                    xaxis=dict(title = 'k-point path [2pi/a]', autorange=True, exponentformat = "e", ticktext=ttext,tickvals=tvals),
                    yaxis=dict(title = 'Energy (ev)', autorange=False, exponentformat = "e", range=[emin, emax],),
                    showlegend=False)                    
            fig = go.Figure(data=data, layout=layout)
            iplot(fig, show_link=False)

    else :
        print ("ERROR: A new connection is required")
        
PARADIM_UI['s3']['button'].on_click(VisualizeBandStructure)
display(PARADIM_UI['s3']['display'])


Group(children=(HTML(value="<p   style='background-color: #DCDCDC; font-size: 150%; padding: 5px'>band-structu…

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><img src='HubzeroLogo.png' style='position:absolute;right:20px;width:110px;height:32px'><h1 style="text-align:center;padding-top:3px">Step 4: Calculate optical absorption coeficient  $\kappa$</h1></div>

We see that the direct gap at $\Gamma$ is $[E_g]$ = 1.42 eV. This value is unusually close to experiment (1.52 eV) for a DFT/LDA calculation. We can also see that in this calculation we have an indirect gap of 1.35 eV along the $\Gamma X$ line. This is an artifact of the DFT/LDA approximation (GaAs is a direct-gap semiconductor).

Now we calculate the imaginary part of the dielectric function, $\epsilon_2(\omega)$. This quantity is related to the optical absorption coeficient $\kappa(\omega)$ by $\kappa(\omega)$ = $\omega \epsilon_2(\omega)/cn(\omega)$, where $\hbar \omega$ is the photon energy, c the speed of light, and n the refractive index.

In order to calculate $\epsilon_2(\omega)$ we use the post-processing code epsilon.x. Before executing epsilon.x we need to perform a new run with pw.x, using a slightly modified input file The modifications are related to the fact that epsilon.x is a fairly basic post-processing code and does not recognize crystal symmetries.

The grid used in the input file includes $10^3$ (reducible) points.

In [6]:
def CalculateCoeficient ( event ):
    global PARADIM_UI, PARADIM
    if PARADIM['SESSION'] is not None and PARADIM['SESSION'].is_active():        
        id_name = PARADIM['TUTORIAL_NAME'] + '_eps'
        pbs_file = id_name + ".pbs"
        inputdeck = PARADIM_UI['s4']['inputb'].value.replace('\n', '\\n')
        PARADIM['SESSION'].execute("printf \"" + inputdeck + "\" > " + id_name + ".in", PARADIM_UI['s4']);
        PARADIM['SESSION'].execute("printf \"" + GetPbs(PARADIM['PBS_TEMPLATE'], id_name, 'epsilon.x').replace('\n', '\\n') + "\" > " + pbs_file, PARADIM_UI['s4']);
        stdin, stdout, stderr, command = PARADIM['SESSION'].execute("qsub " + pbs_file, PARADIM_UI['s4']);
        try:
            code = stdout[len(stdout)-1].strip('\n')
            if (code.isdigit()):
                PARADIM_UI['s4']['job_id'].value = code
                status = 'Q'
                while status != 'E' and status != 'X' and status != 'C' and status != 'O' :
                    status, response, command = getStatus( PARADIM['SESSION'], PARADIM_UI['s4']['job_id'].value, PARADIM_UI['s4'] )
                    PARADIM_UI['s4']['status'].value = status
                    time.sleep(2)
                stdin, stdout, stderr, command = PARADIM['SESSION'].execute("cat " + id_name + ".out", PARADIM_UI['s4']);
                
            else : 
                PARADIM_UI['s4']['job_id'].value = "Error with the PBS submittion"
        except:
            PARADIM_UI['s4']['job_id'].value = "Error with the PBS submittion"
    else :
        print ("ERROR: A new connection is required")


def CalculateSymmetry ( event ):
    global PARADIM_UI, PARADIM
    if PARADIM['SESSION'] is not None and PARADIM['SESSION'].is_active():        
        id_name = PARADIM['TUTORIAL_NAME'] + '_nscf'
        pbs_file = id_name + ".pbs"
        inputdeck = PARADIM_UI['s4']['inputa'].value.replace('\n', '\\n')
        PARADIM['SESSION'].execute("printf \"" + inputdeck + "\" > " + id_name + ".in", PARADIM_UI['s4']);
        PARADIM['SESSION'].execute("printf \"" + GetPbs(PARADIM['PBS_TEMPLATE'], id_name).replace('\n', '\\n') + "\" > " + pbs_file, PARADIM_UI['s4']);
        stdin, stdout, stderr, command = PARADIM['SESSION'].execute("qsub " + pbs_file, PARADIM_UI['s4']);
        try:
            code = stdout[len(stdout)-1].strip('\n')
            if (code.isdigit()):
                PARADIM_UI['s4']['job_id'].value = code
                status = 'Q'
                while status != 'E' and status != 'X' and status != 'C' and status != 'O' :
                    status, response, command = getStatus( PARADIM['SESSION'], PARADIM_UI['s4']['job_id'].value, PARADIM_UI['s4'] )
                    PARADIM_UI['s4']['status'].value = status
                    time.sleep(2)
                stdin, stdout, stderr, command = PARADIM['SESSION'].execute("cat " + id_name + ".out", PARADIM_UI['s4']);
                if (status == 'C'):
                    CalculateCoeficient( event )
                
            else : 
                PARADIM_UI['s4']['job_id'].value = "Error with the PBS submittion"
        except:
            PARADIM_UI['s4']['job_id'].value = "Error with the PBS submittion"
    else :
        print ("ERROR: A new connection is required")

        
PARADIM_UI['s4']['button'].on_click(CalculateSymmetry)
display(PARADIM_UI['s4']['display'])

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><img src='HubzeroLogo.png' style='position:absolute;right:20px;width:110px;height:32px'><h1 style="text-align:center;padding-top:3px">Step 5: Extract Absortion Coeficient  $\kappa$</h1></div>

Previous simulation generates two files that contains the real and imaginary part of the dielectric function, for an electric field polarized along x, y or z. In this case the system is cubic, therefore the x, y and z components will be identical.

In [7]:
def ExtractCoeficients ( stdout ):        
    eps_points = []
    for l in stdout:
        point = [float(s) for s in l.split() if isfloat(s)]
        if len(point) == 4:
            eps_points.append(point)
    return eps_points

def CreateLinePlot( eps_points ):
    data = []
    for key, point in eps_points.items(): 
        x_points = [p[0] for p in point]
        y_points = [p[1] for p in point]
        trace = go.Scatter(
            x = x_points,
            y = y_points,
            mode = 'lines',
            name = key
        )
        data.append(trace)
    return data

def VisualizeCoeficient ( event ):        
    global PARADIM_UI, PARADIM
    if PARADIM['SESSION'] is not None and PARADIM['SESSION'].is_active():
        PARADIM_UI['s5']['output'].clear_output()
        eps_points = {}
        with PARADIM_UI['s5']['output']:            
            stdin, stdout, stderr, command = PARADIM['SESSION'].execute("cat epsr_gaas.dat");
            eps_points['Re(eps)'] = ExtractCoeficients(stdout)
            PARADIM_UI['s5']['input'].value = ' '.join([str(p[0]) for p in eps_points['Re(eps)']])
            stdin, stdout, stderr, command = PARADIM['SESSION'].execute("cat epsi_gaas.dat");
            eps_points['Img(eps)'] = ExtractCoeficients(stdout)
            PARADIM_UI['s5']['input'].value = PARADIM_UI['s5']['input'].value + ' '.join([str(p[0]) for p in eps_points['Img(eps)']])
            data = CreateLinePlot(eps_points)
            layout = go.Layout(title='Electric Field polarization',
                    xaxis=dict(title = 'Rel. diel. function', autorange=True, exponentformat = "e"),
                    yaxis=dict(title = 'Energy (ev)', autorange=True, exponentformat = "e"),
                    showlegend=True,
                    legend=dict(x=0.7, y=1.0)
                    )
            fig = go.Figure(data=data, layout=layout)
            iplot(fig, show_link=False)

    else :
        print ("ERROR: A new connection is required")
        
PARADIM_UI['s5']['button'].on_click(VisualizeCoeficient)
PARADIM_UI['s5']['output'].clear_output()
display(PARADIM_UI['s5']['display'])

Group(children=(HTML(value="<p   style='background-color: #DCDCDC; font-size: 150%; padding: 5px'>band-structu…

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><img src='HubzeroLogo.png' style='position:absolute;right:20px;width:110px;height:32px'><h1 style="text-align:center;padding-top:3px"> Conclusion </h1></div>

In the above plot we can see that the curves are not very smooth. This phenomenon is related to the sampling of the Brillouin zone: by default step 4 used a 10x10x10 mesh, and this is definitely not enough for studying the dielectric function. The meshes required for calculations of dielectric functions may need to contain as many as 50x50x50 points. With the initial coarse 10x10x10 mesh it is difficult to identify the optical  absorption onset around the direct band gap at 1.4 eV. A more accurate calculation using a 30x30x30 together with a zoom would it make it is possible to see the onset around 1.4 eV (broadened by a Gaussian smearing via the input variable intersmear = 0.2 eV)
